In [ ]:
!python -m pip install --upgrade pip setuptools wheel -q
!python -m pip install lbry-libtorrent wget torf -q
!apt install python3-libtorrent -q

In [ ]:
import libtorrent as lt
import time
import os

params = {
    'save_path': '/content/Torrent/',
    'storage_mode': lt.storage_mode_t(2),
}

ses = lt.session()
ses.listen_on(6881, 6891)
link = input("Input Torrent file link or Magnet link and Press Enter: ")

In [ ]:
if link.endswith('.torrent'):
    import wget
    from torf import Torrent

    if os.path.exists('torrent.torrent'):
        os.remove('torrent.torrent')

    wget.download(link, 'torrent.torrent')
    t = Torrent.read('torrent.torrent')
    link = str(t.magnet(name=True, size=False, trackers=False, tracker=False))


print(link)
handle = lt.add_magnet_uri(ses, link, params)

In [ ]:
# Wait for metadata to be available
print("Fetching metadata...")
while not handle.has_metadata():
    time.sleep(1)
print("Metadata fetched!")

In [ ]:
# Print the file list
files = handle.get_torrent_info().files()
for i in range(files.num_files()):
    print(f'File {i}: {files.file_name(i)}, Size: {files.file_size(i)} bytes;')

# <<-- Comment this section to disable google drive saves-->>

In [ ]:

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')
# Import the shutil module
import shutil


# <<<--------------------------------------------------------->>>>

In [8]:
#Creating a File-List Dict
file_info={}
# Iterate over each index i
for i in range(files.num_files()):
    # Retrieve file path and size for each index
    file_name = files.file_name(i)
    file_path = files.file_path(i)

    # Construct the entry for the dictionary
    file_info[f'{i}'] = {
        'name': file_name,
        'path': file_path,
    }

In [ ]:
#Looping Statements For Sequential Download

In [ ]:
import os
import shutil
for key, value in file_info.items():
  # Setting the file priorities
  file_priorities = [0] * files.num_files()  # Set all files to 0 (skip)
  file_priorities[int(key)] = 1  # Set the file to 1 (download)
  handle.prioritize_files(file_priorities)
  # Downloading the Files

  print("File Downloading : " + str(value['name']))
  # Start the session
  ses.start_dht()
  print(value['name'])
  # Monitor the download progress
  while True:
    s = handle.status()
    state_str = ['queued', 'checking', 'downloading metadata',
                 'downloading', 'finished', 'seeding', 'allocating']

    print('%.2f%% complete (down: %.1f kB/s up: %.1f kB/s peers: %d) %s ' %
          (s.progress * 100, s.download_rate / 1000, s.upload_rate / 1000,
           s.num_peers, state_str[s.state]))

    if s.state == lt.torrent_status.finished:
        print(handle.name(), "COMPLETE")
        break

    time.sleep(5)

    end = time.time()
    
    
#<<<<<<<<<<<<<<--- Comment to disable Google Drive saves--->>>>>>>>>>>>>>>>>


  #Copying the Recently Downloaded file to drive
  source_file_path = "/content/Torrent/" + value['path']
  destination_file_path = '/content/drive/My Drive/'

  # Copy the file
  shutil.copy(source_file_path, destination_file_path)

  print('File copied successfully.')
#Deleting the directory after each file copy to facilitate unlimites size downloads

# Define the directory path and the file to keep
  file_path = value['path']
  path_to_delete = '/content/Torrent/'+ file_path
  os.remove(path_to_delete)
  print('deleted' + value['name'])
  
  
#<<<<<<<<<<<<<<<<<<<------------------------------------->>>>>>>>>>>>>>>>>>>>>>>